<a href="https://colab.research.google.com/github/gpriya32/project_elc/blob/main/Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

Requirement already up-to-date: bert-serving-server[http] in /usr/local/lib/python3.6/dist-packages (1.10.0)


In [14]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2021-01-07 05:24:47--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.208, 142.250.73.240, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.2’

uncased_L-12_H-768_ 100%[===================>] 388.84M   129MB/s    in 3.0s    

2021-01-07 05:24:50 (129 MB/s) - ‘uncased_L-12_H-768_A-12.zip.2’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
replace uncased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index

In [15]:
%tensorflow_version 1.x
!nohup bert-serving-start -max_seq_len=1000 -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [16]:
from bert_serving.client import BertClient
bc = BertClient(check_length=False)

## Checking server and dimensions of vector

In [17]:
encoding=bc.encode(['First do it'])
print('Vector length: ',str(len(encoding[0])))

KeyboardInterrupt: ignored

## Unzip the dataset

In [18]:
!unzip COVID_19_dataset.zip

Archive:  COVID_19_dataset.zip
   creating: COVID_19_dataset/
   creating: COVID_19_dataset/documents/
  inflating: COVID_19_dataset/documents/001.txt  
  inflating: COVID_19_dataset/documents/002.txt  
  inflating: COVID_19_dataset/documents/003.txt  
  inflating: COVID_19_dataset/documents/004.txt  
  inflating: COVID_19_dataset/documents/005.txt  
  inflating: COVID_19_dataset/documents/006.txt  
  inflating: COVID_19_dataset/documents/007.txt  
  inflating: COVID_19_dataset/documents/008.txt  
  inflating: COVID_19_dataset/documents/009.txt  
  inflating: COVID_19_dataset/documents/010.txt  
   creating: COVID_19_dataset/summary/
  inflating: COVID_19_dataset/summary/001.txt  
  inflating: COVID_19_dataset/summary/002.txt  
  inflating: COVID_19_dataset/summary/003.txt  
  inflating: COVID_19_dataset/summary/004.txt  
  inflating: COVID_19_dataset/summary/005.txt  
  inflating: COVID_19_dataset/summary/006.txt  
  inflating: COVID_19_dataset/summary/007.txt  
  inflating: COVID_19_

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
import os
import numpy as np
import pandas as pd
import nltk
import re
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
def clean_sent(sentence):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  return sentence

## Read the summary of each document

In [24]:
path='/content/COVID_19_dataset/summary/'
filelist = os.listdir(path)
sentences=[]
cleaned=[]
for file in filelist:
    f = open(path+file, "r",encoding='utf-8',errors='ignore')
    text=f.read()
    sent_tokens = nltk.sent_tokenize(text)
    for sentence in sent_tokens:
      sentences.append(sentence)
      cleaned_sent=clean_sent(sentence)
      cleaned.append(cleaned_sent)

## Get BERT encodings

In [ ]:
bert_encoding=[]
max=0
for sent in sentences:
  sent_encode=bc.encode([sent])
  bert_encoding.append(sent_encode)

##Tabulate all Sentences

In [ ]:
df=pd.DataFrame(sentences,columns=['Sentences'])
df['Cleaned Sentences']=cleaned
df['Encoding']=bert_encoding


In [ ]:
df

In [ ]:
def getAnswer(question):
  ques_encoding=bc.encode([question])
  temp=df.copy()
  sim=[]
  for line in bert_encoding:
    val=cosine_similarity(ques_encoding,line)
    sim.append(val[0][0])
  temp['Similarity']=sim
  temp=temp.sort_values(by='Similarity',ascending=False)
  ans=""
  if temp['Similarity'].iloc[0]>0.90:
    ans=ans+temp['Sentences'].iloc[0]
  elif temp['Similarity'].iloc[0] <0.90:
    c=0
    acc=0
    for i in temp.values:
        ans=ans+i[0]
        c+=1
        acc+=i[3]
        if c==5:
          acc=acc/5
          break
  return ans,acc

In [ ]:
question=''
question=clean_sent(question)
ans,acc=getAnswer(question)
print('Question:',question)
print('Answer:',ans)
print('Avg Accuracy:',acc)

In [ ]:
print(sentences)